In [1]:
import pandas as pd
import numpy as np
import MySQLdb
import os
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from astropy.table import QTable, Table

In [2]:
db = MySQLdb.connect(host='104.154.94.28',db='loganp',\
                     read_default_file="~/.my.cnf",\
                     autocommit=True,\
                     local_infile = 1)
c=db.cursor()

In [3]:
def get_targets(ra,dec,ang_diam,project,name,dist=8000,con=db,database='master_gaia_database'):
    """
    Given an RA/Dec array of any size, returns a Pandas databse of
    the targets from the master database that fall within 
    a circle of specified angular diameter of the given RA/Decs
    All angles in degrees
    Args:
        ra,dec (array, float [deg]): arrays of pointing coordinates in decimal degrees
        project (string): project name that this pointing came from
        name (string): name of the primary beam target
        ang_diam (float [deg]): angular size of the diameter of the beam you are simulating.  Default of 0.8 deg
                          is slightly smaller than the MeerKAT beam in L-band to provide a conservative estimate
        dist (float, [pc]): depth of the desired query in parsecs.  Default is set larger than the largest distance
                          in the master database to return essentially no distance cut
        con (MySQL connection object): the connection you would like to use for the retrieval
        database (str): the database to query.  Default is the master database of all Gaia targets in our program
    Returns:
        Pandas dataframe of every object in the database meeting criteria
    """
    index = range(len(ra))
    appended_data = [] #make a list to store dataframes
    for r,d,i in zip(ra,dec,index):
        string = 'SELECT * FROM '+str(database)+' \
                    WHERE POWER((ra-('+str(r)+')),2) + POWER((decl - ('+str(d)+')),2) < '+str((ang_diam/2.)**2)+' \
                    AND `dist.c` <= '+str(dist)+';'
        dataframe = pd.read_sql(string, con=con)
        dataframe['project']=project
        dataframe['name']=name
        # store DataFrame in list
        appended_data.append(dataframe)
        print "I've done ",i+1," of ",len(ra)," total pointings"
    targets = pd.concat(appended_data, axis=0)
    return targets

def find_exoplanets(ra,dec,ang_diam):
    """
    Given an RA/Dec array of any size, queries the NASA Exoplanet Archive and returns a Pandas databse of
    any known exoplanet hosts that fall within a circle of specified angular diameter of the given RA/Decs
    All angles in degrees
    Args:
        ra,dec (array, float [deg]): arrays of pointing coordinates in decimal degrees
        ang_diam (float [deg]): angular size of the diameter of the beam you are simulating.  Default of 0.8 deg
                          is slightly smaller than the MeerKAT beam in L-band to provide a conservative estimate
    Returns:
        Astropy table object of every confirmed exoplanet host within the circle.
    """
    from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
    from astropy.table import QTable, Table
    # Get exoplanet archive table of all confirmed exoplanets:
    result_table = NasaExoplanetArchive.get_confirmed_planets_table()
    # Convert QTable to regular astropy Table object for easy referencing:
    e = Table(result_table)
    # Convert to pandas df
    cols = e.colnames[0:len(e.colnames)-1]
    ee = e[cols].filled(-9999)
    e = ee.to_pandas()
    
    # Create array for results:
    appended_data = []
    for r,d in zip(ra,dec):
        circle = ((r)-(e['ra']))**2 + ((d)-(e['dec']))**2 <= ang_diam**2
        appended_data.append(e[circle])
    exoplanets = pd.concat(appended_data, axis=0)
    return exoplanets

In [ ]:
# All the pulsars in ATNF catalog within dec -90 & 0:
k=pd.read_csv('../atnf_all_psr_lessthan45dec.csv',delimiter=';',comment='#')
k

In [ ]:
# Get just the binaries:
m=k.loc[np.where((k['PB'].values<'1')&(k['Minimum'].values>='0.3'))[0]]
m=m.reset_index(drop=True)
m

In [ ]:
# Remove any that are in globular clusters, because they will be observed by MeerTime GC program -> Look for
# any associated with GC:
print m['ASSOC']

In [ ]:
m=m.drop(index=[0,1,2,3,4])
print m['ASSOC']

In [ ]:
m.to_csv('MeerTime_binaries_atnfcatalog_notgc.csv')

In [ ]:
string = 'SELECT ra,decl FROM master_gaia_database \
                    WHERE decl BETWEEN -90.0 and 45.0;'
dataframe = pd.read_sql(string, con=db)

In [ ]:
from astropy.coordinates import Angle
mpra = Angle(m['RAJD'].values*u.degree)[0:100]
mpra = mpra.wrap_at(180*u.degree)
mpdec = Angle(m['DECJD'].values*u.degree)[0:100]

hbra = Angle(dataframe['ra'].values*u.degree)
hbra = hbra.wrap_at(180*u.degree)
hbdec = Angle(dataframe['decl'].values*u.degree)

%matplotlib notebook
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
hb = ax.hexbin(hbra.radian, hbdec.radian,gridsize=500,cmap='inferno',bins='log',mincnt=1)
#ax.scatter(pra.radian, pdec.radian,alpha=0.7,color='orange',s=10, label='PPTA pulsars')
ax.scatter(mpra.radian, mpdec.radian,alpha=0.7,color='blue',s=10, label='Binary Targets')
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'],color='grey')
ax.set_title('MeerTime binaries pointings \n')
cb = fig.colorbar(hb, ax=ax,orientation="horizontal", pad=0.1)
cb.set_label('log10(count)')
ax.grid(True)
plt.show()
#plt.legend(fontsize=12,loc=4)
plt.savefig('meertime_binaries_pointings_mollweide.png',format='png',dpi=300)

In [ ]:
# Get all PPTA targets within each pointing:
targets = get_targets(m['RAJD'].values,m['DECJD'].values,ang_diam=0.8)
targets.to_csv('meertime_binaries.csv')

In [4]:
# Import the target list
k = pd.read_csv('MeerTime_binaries_atnfcatalog_notgc.csv')

# Get all targets within each pointing:
count = 0
index = np.arange(0,len(k),1)
for i in index:
        print 'Doing index ',i,' of ',np.max(index)
        targets = get_targets([k['RAJD'].values[i]],[k['DECJD'].values[i]],0.8,'MeerTime Binaries',k['NAME'].values[i])
        if count == 0:
                targets.to_csv('../Our_targets/meertime_binaries.csv',index=False)
        else:
                targets.to_csv('../Our_targets/meertime_binaries.csv',mode='a',header=False,index=False)

        count=count+1

Doing index  0  of  15
I've done  1  of  1  total pointings
Doing index  1  of  15
I've done  1  of  1  total pointings
Doing index  2  of  15
I've done  1  of  1  total pointings
Doing index  3  of  15
I've done  1  of  1  total pointings
Doing index  4  of  15
I've done  1  of  1  total pointings
Doing index  5  of  15
I've done  1  of  1  total pointings
Doing index  6  of  15
I've done  1  of  1  total pointings
Doing index  7  of  15
I've done  1  of  1  total pointings
Doing index  8  of  15
I've done  1  of  1  total pointings
Doing index  9  of  15
I've done  1  of  1  total pointings
Doing index  10  of  15
I've done  1  of  1  total pointings
Doing index  11  of  15
I've done  1  of  1  total pointings
Doing index  12  of  15
I've done  1  of  1  total pointings
Doing index  13  of  15
I've done  1  of  1  total pointings
Doing index  14  of  15
I've done  1  of  1  total pointings
Doing index  15  of  15
I've done  1  of  1  total pointings
